# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import warnings

# Import API key
from api_keys import geoapify_key
warnings.filterwarnings("ignore")

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mangrol,21.1167,70.1167,21.18,85,0,4.43,IN,1707180486
1,1,port-aux-francais,-49.3500,70.2167,6.17,74,100,12.83,TF,1707180486
2,2,searcy,35.2506,-91.7363,13.00,44,0,1.54,US,1707180486
3,3,san patricio,28.0170,-97.5169,17.13,73,0,3.43,US,1707180486
4,4,lebu,-37.6167,-73.6500,16.53,93,98,2.25,CL,1707180486


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    color='City', 
    size='Humidity',
    scale= 0.8,
    title='City Map with Humidity',
    xlabel='Longitude', 
    ylabel='Latitude',
    hover_cols=['City', 'Humidity'],
    geo = True, tiles="OSM")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
#Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (
    (city_data_df['Max Temp'] >= 21) &  #Max temperature greater than or equal to 21 degrees Celsius
    (city_data_df['Max Temp'] <= 27) &  #Max temperature less than or equal to 27 degrees Celsius
    (city_data_df['Humidity'] <= 100) &  #Humidity less than or equal to 100%
    (city_data_df['Wind Speed'] < 4.50) & #Wind Speed less than 4.50
    (city_data_df['Cloudiness'] == 0))  #Example: Cloudiness equals 0%

#Create new DataFrame
ideal_city_df = city_data_df[ideal_conditions]

#Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

#Organize the DataFrame by the 'City_ID' column in ascending order
ideal_city_df = ideal_city_df.sort_values(by='City_ID')

#Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mangrol,21.1167,70.1167,21.18,85,0,4.43,IN,1707180486
220,220,taltal,-25.4000,-70.4833,21.64,82,0,1.48,CL,1707180526
222,222,ciudad lazaro cardenas,17.9583,-102.2000,24.59,77,0,1.42,MX,1707180526
231,231,veraval,20.9000,70.3667,21.45,83,0,4.34,IN,1707180527
279,279,camayenne,9.5350,-13.6878,26.05,89,0,0.00,GN,1707180535
309,309,port saint john's,-31.6229,29.5448,22.54,92,0,2.63,ZA,1707180540
320,320,mermoz boabab,14.7065,-17.4758,24.31,83,0,4.12,SN,1707180542
327,327,al ghayzah,16.2079,52.1760,22.51,68,0,2.38,YE,1707180543
409,409,tintafor,8.6267,-13.2150,25.09,83,0,1.79,SL,1707180558
424,424,lazaro cardenas,17.9583,-102.2000,24.59,77,0,1.42,MX,1707180560


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
#Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

#Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name='')

#Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel_Name
0,mangrol,IN,21.1167,70.1167,85,
220,taltal,CL,-25.4000,-70.4833,82,
222,ciudad lazaro cardenas,MX,17.9583,-102.2000,77,
231,veraval,IN,20.9000,70.3667,83,
279,camayenne,GN,9.5350,-13.6878,89,
309,port saint john's,ZA,-31.6229,29.5448,92,
320,mermoz boabab,SN,14.7065,-17.4758,83,
327,al ghayzah,YE,16.2079,52.1760,68,
409,tintafor,SL,8.6267,-13.2150,83,
424,lazaro cardenas,MX,17.9583,-102.2000,77,


In [8]:
#Loop through the DataFrame Lat and Lng to view the JSON response for the next step.
##NOTE: A lot of this will be repeated in the next step, just to show the format
##in the way it is required for this assigment. I need to view the JSON object to extract the 'keys'
for index, row in hotel_df.iterrows():
    #Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    #Set parameters to search for a hotel
    radius = 10000
    latitude = row['Lat']
    longitude = row['Lng']
    categories = "accommodation.hotel"
    limit = 1
    filter_geometry = f'circle:{longitude},{latitude},{radius}'
    bias = f'proximity:{longitude:.6f},{latitude:.6f}'  # Use 6 decimal places for precision
    params = {
        'categories': categories,
        'limit': limit,
        'filter[geometry]': filter_geometry,
        'bias': bias,
        'apiKey': geoapify_key}  

    #Make an API request using the query URL
    response = requests.get(base_url, params=params)

    #Convert the API response to JSON format
    name_address = response.json()

    #Print the JSON response for each row
    print(f"Hotel information for {row['City']} - {row['Country']}:")
    print(json.dumps(name_address, indent=4, sort_keys=True))
    print("\n")


Hotel information for mangrol - IN:
{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    70.35207026790661,
                    20.934717751156022
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Hotel Park",
                "address_line2": "Veraval Junagadh Road, Veraval - 362200, Gujarat, India",
                "categories": [
                    "accommodation",
                    "accommodation.hotel",
                    "building",
                    "building.accommodation"
                ],
                "city": "Veraval",
                "country": "India",
                "country_code": "in",
                "county": "Patan Veraval Taluka",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
        

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
# Set the radius to 10,000 meters (10 kilometers)
radius = 10000 
params = {
        'categories': categories,
        'limit': limit,
        'filter[geometry]': filter_geometry,
        'bias': bias,
        'apiKey': geoapify_key}  

categories = "accommodation.hotel"
limit = 1
filter_geometry = f'circle:{longitude},{latitude},{radius}'
bias = f'proximity:{longitude},{latitude}' 

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
        hotel_name = name_address["features"][0]["properties"]["address_line1"]
        # Update the 'Hotel_Name' column in the DataFrame
        hotel_df.at[index, "Hotel_Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel_Name']]

Starting hotel search
mangrol - nearest hotel: Hotel Park
taltal - nearest hotel: Hostería Taltal
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
veraval - nearest hotel: Shyam
camayenne - nearest hotel: Palm Camayenne
port saint john's - nearest hotel: Outback Inn
mermoz boabab - nearest hotel: La maison blanche
al ghayzah - nearest hotel: فندق تاج العرب
tintafor - nearest hotel: Sweet Salone
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
bonthe - nearest hotel: Samies
dwarka - nearest hotel: The Dwarika Hotel
kankan - nearest hotel: Buffet de la gare
tshabong - nearest hotel: Van Zyls Rushotel


,City,Country,Lat,Lng,Humidity,Hotel_Name
0,mangrol,IN,21.1167,70.1167,85,Hotel Park
220,taltal,CL,-25.4000,-70.4833,82,Hostería Taltal
222,ciudad lazaro cardenas,MX,17.9583,-102.2000,77,Hotel Sol del Pacífico
231,veraval,IN,20.9000,70.3667,83,Shyam
279,camayenne,GN,9.5350,-13.6878,89,Palm Camayenne
309,port saint john's,ZA,-31.6229,29.5448,92,Outback Inn
320,mermoz boabab,SN,14.7065,-17.4758,83,La maison blanche
327,al ghayzah,YE,16.2079,52.1760,68,فندق تاج العرب
409,tintafor,SL,8.6267,-13.2150,83,Sweet Salone
424,lazaro cardenas,MX,17.9583,-102.2000,77,Hotel Sol del Pacífico


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot2 = ideal_city_df.hvplot.points('Lng', 'Lat', color='City',
                                        size='Humidity',
                                        frame_width=700,
                                        frame_height=500,
                                        title='Ideal Vacation Spot',  
                                        geo=True, 
                                        tiles="OSM", 
                                        hover_cols=['City', 'Country', 'Humidity', 'Hotel_Name'])

# Display the map
map_plot2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)